In [3]:
import pandas as pd
data = pd.read_csv("/workspaces/4900/healthcare-dataset-stroke-data.csv")

balanced_data = data.dropna()
balanced_data = balanced_data.drop(columns= "id")
balanced_data = balanced_data[balanced_data.age > 20]
balanced_data = balanced_data.drop(balanced_data.index[balanced_data["gender"] == "Other"])

features = ['gender', 'age', 'hypertension', 'heart_disease', 'ever_married', 'work_type', 'Residence_type', 'avg_glucose_level', "bmi", "smoking_status"]
cat_columns = ['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status', 'hypertension', 'heart_disease']
num_columns = ['age', 'avg_glucose_level', 'bmi']

In [4]:
from sklearn.model_selection import train_test_split

bal_data_train, bal_data_test = train_test_split(balanced_data, train_size = 0.8, random_state=1)

X_train = bal_data_train.drop('stroke', axis = 1)
y_train = bal_data_train['stroke']

X_test = bal_data_test.drop('stroke', axis = 1)
y_test = bal_data_test['stroke']

In [5]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

preprocessing = ColumnTransformer([
    ('one-hot-encoder', OneHotEncoder(sparse_output=False), cat_columns),
    ('scaler', StandardScaler(), num_columns)
])

SMOTENC TEST

In [6]:
from imblearn.pipeline import Pipeline as imb_pipeline
from sklearn.tree import DecisionTreeClassifier
from imblearn.over_sampling import SMOTENC, RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

smotenc_pipeline = imb_pipeline(steps= [
        ("smotenc", SMOTENC(cat_columns, random_state = 1)),
        ('preprocessing', preprocessing),
        ('model', DecisionTreeClassifier(random_state = 1))
       ])

smotenc_pipeline.fit(X_train, y_train)

smotenc_pipeline_prediction = smotenc_pipeline.predict(X_test)

Over sampler

In [7]:
over_pipeline = imb_pipeline(steps= [
        ("over sampler", RandomOverSampler(random_state= 1)),
        ('preprocessing', preprocessing),
        ('model', DecisionTreeClassifier(random_state = 1))
       ])

over_pipeline.fit(X_train, y_train)

over_pipeline_prediction = over_pipeline.predict(X_test)

Under sampler

In [8]:
under_pipeline = imb_pipeline(steps= [
        ("over sampler", RandomUnderSampler(random_state= 1)),
        ('preprocessing', preprocessing),
        ('model', DecisionTreeClassifier(random_state = 1))
       ])

under_pipeline.fit(X_train, y_train)

under_pipeline_prediction = under_pipeline.predict(X_test)

In [11]:
from sklearn.metrics import recall_score

print("Recall score for smotenc:", recall_score(y_test, smotenc_pipeline_prediction))
print("Recall score for over sampler:", recall_score(y_test, over_pipeline_prediction))
print("Recall score for under sampler:", recall_score(y_test, under_pipeline_prediction))

Recall score for smotenc: 0.2765957446808511
Recall score for over sampler: 0.1276595744680851
Recall score for under sampler: 0.5957446808510638
